## Python Docstrings and Comments Tool

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")
base_url = os.getenv("OPENROUTER_BASE_URL")

if api_key:
    print(f"OpenRouter API Key exists and begins {api_key[:6]}")
else:
    print("OpenRouter API Key not set (and this is optional)")

OpenRouter API Key exists and begins sk-or-


In [3]:
# Connect to client libraries
openrouter = OpenAI(api_key=api_key, base_url=base_url)

In [4]:
models = ["gpt-5", "qwen/qwen3-coder-30b-a3b-instruct" ]

clients = {"gpt-5": openrouter, "qwen/qwen3-coder-30b-a3b-instruct": openrouter}

In [9]:
system_prompt = """
You are an expert Python engineer.

Your job:
- Add Google-style docstrings to every function and class.
- Add concise inline comments explaining non-trivial logic.
- Preserve formatting.
- Do NOT change functionality.
- Return ONLY valid Python code.
"""


def user_prompt_for(python_code: str) -> str:
    return f"""
        Add professional docstrings and inline comments to the following Python code.

        Rules:
        - Do not modify logic.
        - Do not refactor.
        - Do not optimize.
        - Only add documentation.
        - Return valid Python code only.

        Here is the code:

        ```python
        {python_code}
    """


In [10]:
def add_docstrings_and_comments(model, python_code):
    client = clients[model]
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt_for(python_code)}
        ],
    )

    return response.choices[0].message.content

In [11]:
default_code = """
def add(a,b):
    return a+b
"""

with gr.Blocks(title="Python Docstrings + Comments") as ui:
    gr.Markdown(
        "# Python Docstrings and Comments\n"
        "Paste Python code, generate docstrings and helpful comments without changing the code logic."
    )

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_in = gr.Code(label="Python input", value=default_code, language="python", lines=26)
        with gr.Column(scale=6):
            python_out = gr.Code(label="Output (documented)", value="", language="python", lines=26)

    with gr.Row():
        model = gr.Dropdown(models, value=models[0], label="Model")
        generate = gr.Button("Generate")

    generate.click(fn=add_docstrings_and_comments, inputs=[model, python_in], outputs=[python_out])

    ui.launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
